In [90]:
using JuMP, Gurobi, Statistics, Plots, Random, CSV, DataFrames
#project length
t_p = 20
#discount rate
r = 0.06
#Annualization Factor
ϵ = 1 / ((1 - 1 / (1 + r) ^ t_p) / r)

#Dimension limits
D_ceil = 12
D_floor = 2

#products
P = Set(["PB", "PF", "PO", "PG", "ETH", "PPL", "H"])
R = Set(["MSW", "PB", "PF", "PO"])
I = union(Set(P),Set(R))

#locations
G_t = ["B","D"]
G_s = Dict("MSW"=>["B"], "PB"=>["C"], "PF"=>["B"], "PO"=>["D"])
G_d = Dict("PB"=>["C"], "PF"=>["D"], "PO"=>["B"], "ETH"=>["C"], "PPL"=>["B"], "H"=>["B"])

#technologies
T = Dict("{PB}_{MSW}_t1"=>[Dict("Omega"=>["PB"]), Dict("Kappa"=>["MSW"]), Dict("Theta"=>"t1")], 
    "{PB}_{MSW}_t2"=>[Dict("Omega"=>["PB"]), Dict("Kappa"=>["MSW"]), Dict("Theta"=>"t2")], 
    "{PB}_{MSW}_t3"=>[Dict("Omega"=>["PB"]), Dict("Kappa"=>["MSW"]), Dict("Theta"=>"t3")],
    "{PF}_{PB}_t1"=>[Dict("Omega"=>["PF"]), Dict("Kappa"=>["PB"]), Dict("Theta"=>"t1")], 
    "{PF}_{PB}_t2"=>[Dict("Omega"=>["PF"]), Dict("Kappa"=>["PB"]), Dict("Theta"=>"t2")], 
    "{PF}_{PB}_t3"=>[Dict("Omega"=>["PF"]), Dict("Kappa"=>["PB"]), Dict("Theta"=>"t3")], 
    "{PO,PG}_{PF}_t1"=>[Dict("Omega"=>["PO", "PG"]), Dict("Kappa"=>["PF"]), Dict("Theta"=>"t1")], 
    "{PO,PG}_{PF}_t2"=>[Dict("Omega"=>["PO", "PG"]), Dict("Kappa"=>["PF"]), Dict("Theta"=>"t2")], 
    "{PO,PG}_{PF}_t3"=>[Dict("Omega"=>["PO", "PG"]), Dict("Kappa"=>["PF"]), Dict("Theta"=>"t3")], 
    "{ETH,PPL,H}_{PO}_t1"=>[Dict("Omega"=>["ETH", "PPL", "H"]), Dict("Kappa"=>["PO"]), Dict("Theta"=>"t1")],
    "{ETH,PPL,H}_{PO}_t2"=>[Dict("Omega"=>["ETH", "PPL", "H"]), Dict("Kappa"=>["PO"]), Dict("Theta"=>"t2")],
    "{ETH,PPL,H}_{PO}_t3"=>[Dict("Omega"=>["ETH", "PPL", "H"]), Dict("Kappa"=>["PO"]), Dict("Theta"=>"t2")])
    ## products of each technology
Ω_t = Dict()
for t in keys(T)
    Ω_t[t] = values(T[t][1]["Omega"])
end
    ## feed of each technology
Κ_t = Dict()
for t in keys(T)
    Κ_t[t] = values(T[t][2]["Kappa"])
end
    ## types of each technology
Θ_t = Dict() # types of each technology
for t in keys(T)
    Θ_t[t] = values(T[t][3]["Theta"])
end
    ##capacity of technology
ξ_t = Dict("{PB}_{MSW}_t1"=>Dict("PB"=>241800), "{PB}_{MSW}_t2"=>Dict("PB"=>604500), "{PB}_{MSW}_t3"=>Dict("PB"=>1209000), 
    "{PF}_{PB}_t1"=>Dict("PF"=>208000), "{PF}_{PB}_t2"=>Dict("PF"=>520000), "{PF}_{PB}_t3"=>Dict("PF"=>1040000),
    "{PO,PG}_{PF}_t1"=>Dict("PO"=>169243, "PG"=>39420), "{PO,PG}_{PF}_t2"=>Dict("PO"=>423108, "PG"=>98550), "{PO,PG}_{PF}_t3"=>Dict("PO"=>1128288, "PG"=>262800), 
    "{ETH,PPL,H}_{PO}_t1"=>Dict("ETH"=>130743, "PPL"=>80794, "H"=>3980),"{ETH,PPL,H}_{PO}_t2"=>Dict("ETH"=>261486, "PPL"=>161588, "H"=>7960),"{ETH,PPL,H}_{PO}_t3"=>Dict("ETH"=>522972, "PPL"=>323174, "H"=>15920))
    ##dimension
γ_t = Dict("{PB}_{MSW}_t1"=>3, "{PB}_{MSW}_t2"=>5, "{PB}_{MSW}_t3"=>8, "{PF}_{PB}_t1"=>3,  "{PF}_{PB}_t2"=>5, "{PF}_{PB}_t3"=>8,"{PO,PG}_{PF}_t1"=>2, "{PO,PG}_{PF}_t2"=>4, "{PO,PG}_{PF}_t3"=>8, 
    "{ETH,PPL,H}_{PO}_t1"=>3, "{ETH,PPL,H}_{PO}_t2"=>5, "{ETH,PPL,H}_{PO}_t3"=>8)
    ##Installation cost
α_ξ = Dict("{PB}_{MSW}_t1"=>Dict("B"=>26764523.13*0.9, "D"=>26764523.13*1.1), 
    "{PB}_{MSW}_t2"=>Dict("B"=>46379227*0.9,"D"=>46379227*1.1), 
    "{PB}_{MSW}_t3"=>Dict("B"=>70297764*0.9,"D"=>70297764*1.1), 
    "{PF}_{PB}_t1"=>Dict("B"=>45687062.21*1, "D"=>45687062.21*1.1),
    "{PF}_{PB}_t2"=>Dict("B"=>79169378*1,"D"=>79169378*1.1),
    "{PF}_{PB}_t3"=>Dict("B"=>119998339*1,"D"=>119998339*1.1), 
    "{PO,PG}_{PF}_t1"=>Dict("B"=>86381000*1.1, "D"=>86381000*1),
    "{PO,PG}_{PF}_t2"=>Dict("B"=>149734836*1.1,"D"=>149734836*1),
    "{PO,PG}_{PF}_t3"=>Dict("B"=>269600000*1.1, "D"=>269600000*1), 
    "{ETH,PPL,H}_{PO}_t1"=>Dict("B"=>604900000*0.9, "D"=>604900000*1.1), 
    "{ETH,PPL,H}_{PO}_t2"=>Dict("B"=>917000000*0.9,"D"=>917000000*1.1), 
    "{ETH,PPL,H}_{PO}_t3"=>Dict("B"=>1390000000*0.9, "D"=>1390000000*1.1))
    ##Operating cost
α_o = Dict("{PB}_{MSW}_t1"=>Dict("B"=>8.87*1860000*0.9, "D"=>8.87*1860000*1.1), 
    "{PB}_{MSW}_t2"=>Dict("B"=>8.87*4650000*0.9,"D"=>8.87*4650000*1.1), 
    "{PB}_{MSW}_t3"=>Dict("B"=>8.87*9300000*0.9,"D"=>8.87*9300000*1.1), 
    "{PF}_{PB}_t1"=>Dict("B"=>44.19*208000*1, "D"=>44.19*208000*1.1),
    "{PF}_{PB}_t2"=>Dict("B"=>44.19*520000*1,"D"=>44.19*520000*1.1),
    "{PF}_{PB}_t3"=>Dict("B"=>44.19*1040000*1,"D"=>44.19*1040000*1.1), 
    "{PO,PG}_{PF}_t1"=>Dict("B"=>14*219000*1.1, "D"=>14*219000*1),
    "{PO,PG}_{PF}_t2"=>Dict("B"=>14*547500*1.1,"D"=>14*547500*1),
    "{PO,PG}_{PF}_t3"=>Dict("B"=>14*1460000*1.1, "D"=>14*1460000*1), 
    "{ETH,PPL,H}_{PO}_t1"=>Dict("B"=>71.8*497500*0.9, "D"=>71.8*497500*1.1), 
    "{ETH,PPL,H}_{PO}_t2"=>Dict("B"=>71.8*995000*0.9,"D"=>71.8*995000*1.1), 
    "{ETH,PPL,H}_{PO}_t3"=>Dict("B"=>71.8*1990000*0.9, "D"=>71.8*1990000*1.1))

#Dependency
β = Dict( "MSW"=>Dict(),
    "PB"=>Dict("{PB}_{MSW}_t1"=>Dict("MSW"=>1/0.13, "PB"=>0, "PF"=>0, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0), 
        "{PB}_{MSW}_t2"=>Dict("MSW"=>1/0.13, "PB"=>0, "PF"=>0, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0),
        "{PB}_{MSW}_t3"=>Dict("MSW"=>1/0.13, "PB"=>0, "PF"=>0, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0)),
    "PF"=>Dict("{PF}_{PB}_t1"=>Dict("MSW"=>0, "PB"=>1, "PF"=>0, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0), 
        "{PF}_{PB}_t2"=>Dict("MSW"=>0, "PB"=>1, "PF"=>0, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0),
        "{PF}_{PB}_t3"=>Dict("MSW"=>0, "PB"=>1, "PF"=>0, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0)),
    "PO"=>Dict("{PO,PG}_{PF}_t1"=>Dict("MSW"=>0, "PB"=>0, "PF"=>1/0.7728, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0), 
        "{PO,PG}_{PF}_t2"=>Dict("MSW"=>0, "PB"=>0, "PF"=>1/0.7728, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0),
        "{PO,PG}_{PF}_t3"=>Dict("MSW"=>0, "PB"=>0, "PF"=>1/0.7728, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0)),
    "PG"=>Dict("{PO,PG}_{PF}_t1"=>Dict("MSW"=>0, "PB"=>0, "PF"=>1/0.18, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0), 
        "{PO,PG}_{PF}_t2"=>Dict("MSW"=>0, "PB"=>0, "PF"=>1/0.18, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0),
        "{PO,PG}_{PF}_t3"=>Dict("MSW"=>0, "PB"=>0, "PF"=>1/0.18, "PO"=>0, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0)),
    "ETH"=>Dict("{ETH,PPL,H}_{PO}_t1"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.2628, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0), 
        "{ETH,PPL,H}_{PO}_t2"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.2628, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0),
        "{ETH,PPL,H}_{PO}_t3"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.2628, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0)),
    "PPL"=>Dict("{ETH,PPL,H}_{PO}_t1"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.1624, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0), 
        "{ETH,PPL,H}_{PO}_t2"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.1624, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0),
        "{ETH,PPL,H}_{PO}_t3"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.1624, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0)),
    "H"=>Dict("{ETH,PPL,H}_{PO}_t1"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.008, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0), 
        "{ETH,PPL,H}_{PO}_t2"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.008, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0),
        "{ETH,PPL,H}_{PO}_t3"=>Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>1/0.008, "PG"=>0, "ETH"=>0, "PPL"=>0, "H"=>0))
)

#final product
δ = Dict("MSW"=>0, "PB"=>0, "PF"=>0, "PO"=>0, "PG"=>0, "ETH"=>150000, "PPL"=>100000, "H"=>5000)
#purchasing cost
α_ρ = Dict("MSW"=>0, "PB"=>250, "PF"=>1300, "PO"=>1100)
#disposal cost
α_d = Dict("MSW"=>50, "PB"=>40, "PF"=>40, "PO"=>400, "PG"=>800, "ETH"=>0, "PPL"=>0, "H"=>0)
#connectivity cost
π_f = Dict("MSW"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)),
    "PB"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)),
    "PF"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)),
    "PO"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)),
    "PG"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)),
    "ETH"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)),
    "PPL"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)),
    "H"=>Dict("B"=>Dict("B"=>0.01,"C"=>0.1,"D"=>0.14),"C"=>Dict("B"=>0.14,"C"=>0.01,"D"=>0.12),"D"=>Dict("B"=>0.14,"C"=>0.11,"D"=>0.01)));

In [81]:
#Solve for the hierarchy of the products
m = Model(optimizer_with_attributes(Gurobi.Optimizer))
@variable(m, x[I] >= 1, Int)
@constraint(m, [t in keys(T), i in Ω_t[t], ii in Κ_t[t]], x[i] <= x[ii]-1)
@objective(m, Min, sum(x))
status = optimize!(m)
seqq = zeros(length(I))
for i in 1:length(I)
    seqq[i] = value.(x[collect(I)[i]])
end
println()
for i in 1:length(I)
    println(collect(I)[i], " has hierarchy ", seqq[i])
end
I = collect(I)[sortperm(seqq)];

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 21 rows, 8 columns and 42 nonzeros
Model fingerprint: 0x1a398922
Variable types: 0 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 21 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 18 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.800000000000e+01, best bound 1.800000000000e+01, gap 0.0000%

PO has hierarchy 2.0
PB has hierarchy 4.0
ETH has hierarchy 1.0
PG has hierarchy 1.0
PPL has hierarchy 1.0
MSW has hierarchy 5.0
H has hierarchy 1.0
PF has hierarchy 3.0


In [82]:
#Calculating the total amount of each materials required for the process
need = Dict()
all_need = Dict()
for i in I
    all_need[i] = δ[i]
end
needd = Dict()
for iii in I
    needd[iii] = 0
    for tt in keys(T)
        for t in keys(T)
            need[t] = Dict()
            for i in I
                if all_need[i] > 0
                    for ii in Κ_t[t]
                        if (t in keys(β[i]))
                            if (ii ∉ keys(need[t]))
                                need[t][ii] = β[i][t][ii] * all_need[i]
                            elseif (need[t][ii] < β[i][t][ii] * all_need[i])
                                need[t][ii] = β[i][t][ii] * all_need[i]
                            end
                        end
                    end
                end
            end
        end
        if iii in keys(need[tt])
            needd[iii] = max(need[tt][iii], needd[iii])
        end
    end
    all_need[iii] += needd[iii]
end
all_need

Dict{Any,Any} with 8 entries:
  "PO"  => 625000.0
  "ETH" => 150000
  "PB"  => 8.08747e5
  "PG"  => 0
  "PPL" => 100000
  "MSW" => 6.22113e6
  "H"   => 5000
  "PF"  => 8.08747e5

In [83]:
#Generating nodes and their corresponding attributes
S_q = [] # set for supplier nodes
ϕ_u = Dict() # location of node
Ω_u = Dict() # products of node
γ_u = Dict() # dimension of node
ξ_u = Dict() # capacity of node
Κ_u = Dict() # feed of node
τ_u = Dict() # tech of node
for i in I
    if i in R
        for l in G_s[i]
            S_q = push!(S_q,string("s_", i, "_", l))
            ϕ_u[S_q[end]] = l
            Ω_u[S_q[end]] = [i]
            γ_u[S_q[end]] = 0
            Κ_u[S_q[end]] = []
        end
    end
end

D_q = [] # set for demand ndoes
for i in P
    if δ[i]!= 0
        for l in G_d[i]
            D_q = push!(D_q,string("d_", i, "_", l))
            ϕ_u[D_q[end]] = l
            Κ_u[D_q[end]] = [i]
            γ_u[D_q[end]] = 0
            Ω_u[D_q[end]] = []
        end
    end
end

U_q = [] # set for tech nodes
η_u = Dict() # copy of tech of node
θ_u = Dict() # type of tech of node
for t in keys(T)
    for i in Ω_t[t]
        for k in 1:Int(ceil(all_need[i] / ξ_t[t][i]))
            for l in G_t
                if string("u_", t, "_", k, "_", l) ∉ U_q
                    U_q = push!(U_q,string("u_", t, "_", k, "_", l))
                end
                ϕ_u[U_q[end]] = l
                η_u[U_q[end]] = k
                θ_u[U_q[end]] = Θ_t[t]
                γ_u[U_q[end]] = γ_t[t]
                ξ_u[U_q[end]] = ξ_t[t][i]
                Κ_u[U_q[end]] = Κ_t[t]
                τ_u[U_q[end]] = t
                Ω_u[U_q[end]] = Ω_t[t]
            end
        end
    end
end
N_q = [S_q;D_q;U_q]; # set for all nodes

In [84]:
#constructing the spatial superstructure using dependencies
super_stru = zeros(length(N_q), length(N_q))
super_mat = Dict()
aa =[S_q;U_q]
for k in N_q
    super_mat[k] = Dict()
    for kk in N_q
        super_mat[k][kk] = 0          
    end
end

for i in 1:length(D_q)
    for j in 1:length(U_q) + length(S_q)
        for ii in Ω_u[aa[j]]
            if Κ_u[D_q[i]][1] == ii
                super_stru[length(D_q)+j, i] = 1
                super_mat[aa[j]][D_q[i]] = 1
            end
        end
    end
end

for j in 1:length(U_q)
    for i in Κ_u[U_q[j]]
        for k in 1:length(U_q)
            for ii in Ω_u[U_q[k]]
                if i == ii
                    super_stru[length(D_q)+length(S_q)+k, length(D_q)+length(S_q)+j] = 1
                    super_mat[U_q[k]][U_q[j]] = 1
                end
            end
        end
        for kk in 1:length(S_q)
            for ii in Ω_u[S_q[kk]]
                if i == ii
                    super_stru[length(D_q)+kk,length(D_q)+length(S_q)+j] = 1
                    super_mat[S_q[kk]][U_q[j]] = 1
                end
            end
        end
    end
end

In [85]:
#output the edge list and node attributes for spatial superstructure (used to generate plots)
#edge list
source = []
outlet = []
for i in N_q
    for j in N_q
        if super_mat[i][j] == 1
            append!(source,[i])
            append!(outlet,[j])
        end
    end
end
data = DataFrame(source=source,outlet=outlet);
CSV.write("edgelist_spatialsuper_case.csv",  data, writeheader=false)

#node attributes
type_node = []
material_type = []
tech_type = []
copy = []
for i in N_q
    if occursin("s_", i)
        append!(type_node,["buy"])
        append!(material_type, [Ω_u[i]])
        append!(copy, ["-1"])
        append!(tech_type,0)
    elseif occursin("d_", i)
        append!(type_node,["pro"])
        append!(material_type, [Κ_u[i]])
        append!(copy, ["-1"])
        append!(tech_type,0)
    else
        append!(type_node,["tech"])
        if "PO" in Ω_u[i]
            append!(material_type, [["PO"]])
        else
            append!(material_type, [Ω_u[i]])
        end
        append!(copy,i[end-2])
        if occursin("t1", i)
            append!(tech_type,1)
        elseif occursin("t2", i)
            append!(tech_type,2)
        elseif occursin("t3", i)
            append!(tech_type,3)
        end
        
    end
end
data = DataFrame(ID=N_q,type_node=type_node, material=material_type, tech=tech_type, copy=copy);
CSV.write("nodeattr_spatialsuper_case.csv",  data, writeheader=true)

"node_attr_ss_casereal.csv"

In [86]:
#cost minimizing optimal design (without modularity measure)
M = 1e10
m = Model(optimizer_with_attributes(Gurobi.Optimizer))

@variable(m, f[N_q, N_q] >= 0)
@variable(m, y[U_q], Bin)
@variable(m, v[S_q] >= 0)

#Connectivity governed by the spatial superstructure
@constraint(m, [k in N_q, kk in N_q], f[k,kk] <= M*super_mat[k][kk])
#Material balances for raw materials, technology node and final demand
@constraint(m, [s in S_q, i in Ω_u[s]], sum(f[s,k] for k in N_q) <= v[s])
@constraint(m, [u in U_q, i in Ω_u[u]], sum(f[u,k] for k in N_q if i in Κ_u[k]) <= ξ_t[τ_u[u]][i] * y[u])
@constraint(m, [u in U_q, i in Κ_u[u], ii in Ω_u[u]], sum(f[k,u] for k in N_q) == ξ_t[τ_u[u]][ii] * β[ii][τ_u[u]][i] * y[u])
@constraint(m, [d in D_q, i in Κ_u[d]], sum(f[k,d] for k in N_q) == δ[i])

@objective(m, Min, sum((α_ξ[τ_u[u]][ϕ_u[u]] * ϵ + α_o[τ_u[u]][ϕ_u[u]]) * y[u] for u in U_q) + sum(α_ρ[i] * v[s] for s in S_q for i in Ω_u[s]) 
    + sum(π_f[i][ϕ_u[u]][ϕ_u[k]] * sum(f[u,k] for k in N_q) for u in U_q for i in Ω_u[u]) 
    + sum(π_f[i][ϕ_u[s]][ϕ_u[k]] * sum(f[s,k] for k in N_q) for s in S_q for i in Ω_u[s]) 
    + sum((ξ_t[τ_u[u]][i] * y[u] - sum(f[u,k] for k in N_q if i in Κ_u[k])) * α_d[i] for u in U_q for i in Ω_u[u]) 
    + sum((v[s] - sum(f[s,kk] for kk in N_q)) * α_d[i] for s in S_q for i in Ω_u[s]))

status = optimize!(m)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3416 rows, 3303 columns and 8900 nonzeros
Model fingerprint: 0xba7ce25e
Variable types: 3253 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+06]
  Objective range  [1e-02, 7e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3335 rows and 2909 columns
Presolve time: 0.00s
Presolved: 81 rows, 394 columns, 766 nonzeros
Variable types: 354 continuous, 40 integer (40 binary)

Root relaxation: objective 3.384523e+08, 74 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.3845e+08    0    4          - 

In [87]:
#quick look at the result
for k in U_q
    if value.(y[k]) >= 0.1
        println("node ", k, " is installed")
    end
end

node u_{PO,PG}_{PF}_t3_1_D is installed
node u_{PB}_{MSW}_t1_2_B is installed
node u_{PF}_{PB}_t3_1_B is installed
node u_{PB}_{MSW}_t3_1_B is installed
node u_{ETH,PPL,H}_{PO}_t2_1_B is installed
node u_{PF}_{PB}_t1_2_B is installed
node u_{PF}_{PB}_t1_4_B is installed


In [88]:
#optimal design considers both cost and modularity measure
M = 1e20
n = 1:4
m = Model(optimizer_with_attributes(Gurobi.Optimizer))
@variable(m, f[N_q, N_q] >= 0)
@variable(m, a[N_q, N_q], Bin)
@variable(m, y[U_q, n], Bin)
@variable(m, z[n,L], Bin)
@variable(m, a_m[U_q, U_q, n], Bin)
@variable(m, 0 <= v[S_q])

#connectivity governed by the spatial superstructure
@constraint(m, [k in N_q, kk in N_q], f[k,kk] <= M * super_mat[k][kk])
#material balance for raw material, technology and demand nodes
@constraint(m, [s in S_q, i in Ω_u[s]], sum(f[s,k] for k in N_q) <= v[s])
@constraint(m, [u in U_q, i in Ω_u[u]], sum(f[u,k] for k in N_q if i in Κ_u[k]) <= ξ_t[τ_u[u]][i] * sum(y[u,nn] for nn in n))
@constraint(m, [u in U_q, i in Κ_u[u], ii in Ω_u[u]], sum(f[k,u] for k in N_q) == ξ_t[τ_u[u]][ii] * β[ii][τ_u[u]][i]*sum(y[u, nn] for nn in n))
@constraint(m, [d in D_q, i in Κ_u[d]], sum(f[k,d] for k in N_q) == δ[i])
#logical constraint between the flow matrix and its corresponding adjacency matrix
@constraint(m, [k in N_q, kk in N_q], f[k,kk] <= M * a[k,kk])
@constraint(m, [k in N_q,kk in N_q], a[k,kk] <= f[k,kk])
#other logical constraints related to modularity
@constraint(m, [u in U_q], sum(y[u,nn] for nn in n) <= 1)
@constraint(m, [nn in n], sum(z[nn,l] for l in L) <= 1)
@constraint(m, [u in U_q, nn in n], y[u,nn] <= z[nn,ϕ_u[u]])
@constraint(m, [u in U_q, nn in n], sum(a_m[u,uu,nn] for uu in U_q) + sum(a_m[uu,u,nn] for uu in U_q) <= M * y[u, nn])
@constraint(m, [u in U_q, uu in U_q], sum(a_m[u,uu,nn] for nn in n) <= a[u,uu])
#dimensionality constraint
@constraint(m, [nn in n], sum(y[u,nn]*γ_u[u] for u in U_q) <= D_ceil)
@constraint(m, [nn in n], sum(y[u,nn]*γ_u[u] for u in U_q) >= D_floor)
#modularity constraint
@constraint(m, sum(a_m) >= 0.3*sum(a[u,uu] for u in U_q for uu in U_q))

#slack constraints that used to accelerate the solver
@constraint(m, [k in N_q], f[k,k] == 0)
@constraint(m, [k in N_q], a[k,k] == 0)
@constraint(m, [u in U_q, nn in n], a_m[u,u,nn] == 0)
for k in N_q
    for kk in N_q
        for i in Κ_u[kk]
            if i ∉ Ω_u[k]
                @constraint(m, f[k, kk] == 0)
                @constraint(m, a[k, kk] == 0)
            end
        end
    end
end
for u in U_q
    for uu in U_q
        for i in Κ_u[uu]
            if i ∉ Ω_u[u]
                @constraint(m, [nn in n], a_m[u,uu,nn] == 0)
            end
        end
    end
end
for i in 1:length(U_q)
    for ii in i+1:length(U_q)
        if (Ω_u[U_q[i]] == Ω_u[U_q[ii]]) & (Κ_u[U_q[i]] == Κ_u[U_q[ii]]) & (θ_u[U_q[i]] == θ_u[U_q[ii]]) & (η_u[U_q[i]] == η_u[U_q[ii]])
            @constraint(m, [k in N_q], a[k,U_q[i]] + a[k,U_q[ii]] <=1)
            @constraint(m, [k in N_q], a[U_q[i],k] + a[U_q[ii],k] <=1)
            @constraint(m, [u in U_q, nn in n], a_m[u,U_q[i],nn] + a_m[u,U_q[ii],nn] <=1)
            @constraint(m, [u in U_q, nn in n], a_m[U_q[i],u,nn] + a_m[U_q[ii],u,nn] <=1)
            @constraint(m, [nn in n], y[U_q[i],nn] + y[U_q[ii],nn] <= 1)
        end
    end
end

@objective(m, Min, sum((α_ξ[τ_u[u]][ϕ_u[u]] * ϵ + α_o[τ_u[u]][ϕ_u[u]])*sum(y[u,nn] for nn in n) for u in U_q) + sum(α_ρ[i] * v[s] for s in S_q for i in Ω_u[s]) 
    + sum(π_f[i][ϕ_u[u]][ϕ_u[k]] * sum(f[u,k] for k in N_q) for u in U_q for i in Ω_u[u]) 
    + sum(π_f[i][ϕ_u[s]][ϕ_u[k]] * sum(f[s,k] for k in N_q) for s in S_q for i in Ω_u[s]) 
    + sum((ξ_t[τ_u[u]][i] * sum(y[u,nn] for nn in n) - sum(f[u,k] for k in N_q if i in Κ_u[k])) * α_d[i] for u in U_q for i in Ω_u[u]) 
    + sum((v[s]-sum(f[s,kk] for kk in N_q)) * α_d[i] for s in S_q for i in Ω_u[s]))

status = optimize!(m)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 41083 rows, 16710 columns and 111612 nonzeros
Model fingerprint: 0xdb601221
Variable types: 3253 continuous, 13457 integer (13457 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+20]
  Objective range  [1e-02, 7e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+20]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 38454 rows and 15266 columns
Presolve time: 0.26s
Presolved: 2629 rows, 1444 columns, 10094 nonzeros
Variable types: 360 continuous, 1084 integer (1084 binary)
Found heuristic solution: objective 5.105666e+09
Found heuristic solution: objective 2.546097e+09
Found heuristic solution: objective 1.397075e+09

Root relaxation: objective 3.394372e+08, 1623 iterations, 0.08 seconds

    Nodes    |    Current 

In [89]:
#quick look at the result
for k in U_q
    for nn in n
        if value.(y[k,nn]) >= 0.9
            println("node ", k, " is installed in module ", nn)
        end
    end
end

node u_{PO,PG}_{PF}_t3_1_D is installed in module 3
node u_{PB}_{MSW}_t1_4_B is installed in module 4
node u_{PF}_{PB}_t3_1_B is installed in module 4
node u_{PB}_{MSW}_t3_1_B is installed in module 2
node u_{ETH,PPL,H}_{PO}_t2_1_B is installed in module 1
node u_{PF}_{PB}_t1_1_B is installed in module 2
node u_{PF}_{PB}_t1_4_D is installed in module 3


In [32]:
#output the edge list and node attributes of the optimal process design (used to generate plots)
#edge list
source = []
outlet = []
for i in N_q
    for j in N_q
        if value.(a[i,j]) >= 0.8
            append!(source,[i])
            append!(outlet,[j])
        end
    end
end
data = DataFrame(source=source,outlet=outlet);
CSV.write("edgelist_spatialsuper_case_modular_0.csv",  data, writeheader=false)

#node attributes
type_node = []
material_type = []
tech_type = []
copy = []
nodes = []
for i in N_q
    for j in N_q
        if value.(a[i,j]) >= 0.8
            push!(nodes,i)
            push!(nodes,j)
        end
    end
end
nodes = Set(nodes)
nodes = [i for i in nodes]
for i in nodes
    if occursin("s_", i)
        append!(type_node,["buy"])
        append!(material_type, [Ω_u[i]])
        append!(copy, ["-1"])
        append!(tech_type,0)
    elseif occursin("d_", i)
        append!(type_node,["pro"])
        append!(material_type, [Κ_u[i]])
        append!(copy, ["-1"])
        append!(tech_type,0)
    else
        append!(type_node,["tech"])
        if "PO" in Ω_u[i]
            append!(material_type, [["PO"]])
        else
            append!(material_type, [Ω_u[i]])
        end
        append!(copy,i[end-2])
        if occursin("t1", i)
            append!(tech_type,1)
        elseif occursin("t2", i)
            append!(tech_type,2)
        elseif occursin("t3", i)
            append!(tech_type,3)
        end
        
    end
end
data = DataFrame(ID=nodes,type_node=type_node, material=material_type, tech=tech_type, copy=copy);
CSV.write("nodeattr_spatialsuper_case_modular_0.csv",  data, writeheader=true)